In [1]:
!pip install  transformers==4.17 datasets datasets evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.4 MB/s eta 0:00:00


In [3]:
!pip install huggingface_hub

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from datasets import load_dataset

books = load_dataset("opus_books", "en-fr")

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [6]:
books = books["train"].train_test_split(test_size=0.2)

In [7]:
books["train"][0]

{'id': '20128',
 'translation': {'en': 'I descended a little on the side of that delicious vale, surveying it with a secret kind of pleasure, though mixed with my other afflicting thoughts, to think that this was all my own; that I was king and lord of all this country indefensibly, and had a right of possession; and if I could convey it, I might have it in inheritance as completely as any lord of a manor in England.',
  'fr': "Je descendis un peu sur le coteau de cette délicieuse vallée, la contemplant et songeant, avec une sorte de plaisir secret,--quoique mêlé de pensées affligeantes,--que tout cela était mon bien, et que j'étais Roi et Seigneur absolu de cette terre, que j'y avais droit de possession, et que je pouvais la transmettre comme si je l'avais eue en héritance, aussi incontestablement qu'un lord d'Angleterre son manoir."}}

In [8]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [9]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [12]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir="English_to_French",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/barghavani/English_to_French into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/231M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.55k/3.55k [00:00<?, ?B/s]

Download file runs/Nov20_13-39-03_758d098394a7/events.out.tfevents.1700487548.758d098394a7.534.0: 100%|#######…

Download file runs/Nov20_13-39-03_758d098394a7/1700487548.8650837/events.out.tfevents.1700487548.758d098394a7.…

...

Clean file pytorch_model.bin:   0%|          | 1.00k/231M [00:00<?, ?B/s]

Using amp half precision backend
The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation, id. If translation, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 101668
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6355


Epoch,Training Loss,Validation Loss
1,1.887400,1.667305


Saving model checkpoint to English_to_French/checkpoint-500
Configuration saved in English_to_French/checkpoint-500/config.json
Model weights saved in English_to_French/checkpoint-500/pytorch_model.bin
tokenizer config file saved in English_to_French/checkpoint-500/tokenizer_config.json
Special tokens file saved in English_to_French/checkpoint-500/special_tokens_map.json
tokenizer config file saved in English_to_French/tokenizer_config.json
Special tokens file saved in English_to_French/special_tokens_map.json
Saving model checkpoint to English_to_French/checkpoint-1000
Configuration saved in English_to_French/checkpoint-1000/config.json
Model weights saved in English_to_French/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in English_to_French/checkpoint-1000/tokenizer_config.json
Special tokens file saved in English_to_French/checkpoint-1000/special_tokens_map.json
tokenizer config file saved in English_to_French/tokenizer_config.json
Special tokens file saved in Engli

TrainOutput(global_step=6355, training_loss=1.933902536987977, metrics={'train_runtime': 1129.1647, 'train_samples_per_second': 90.038, 'train_steps_per_second': 5.628, 'total_flos': 2500731612954624.0, 'train_loss': 1.933902536987977, 'epoch': 1.0})

In [14]:
from transformers import pipeline

In [ ]:
!pip install -q gradio
!pip install -q  transformers==4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.6/381.6 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.4 MB/

In [25]:
translator_en_French = pipeline("translation", model="English_to_French")



loading configuration file English_to_French/config.json
Model config T5Config {
  "_name_or_path": "English_to_French",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate

In [26]:
translator_en_German = pipeline("translation", model="barghavani/English_to_German")


https://huggingface.co/barghavani/English_to_German/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpd54gky6e


Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

storing https://huggingface.co/barghavani/English_to_German/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/10be16d0e5e5e3685ad1338c13bf0f5e023e101a23ffac075358811959890594.d1b447d2d77f64965845e2315488ade5473578846e926f3195a1a9e440231045
creating metadata file for /root/.cache/huggingface/transformers/10be16d0e5e5e3685ad1338c13bf0f5e023e101a23ffac075358811959890594.d1b447d2d77f64965845e2315488ade5473578846e926f3195a1a9e440231045
loading configuration file https://huggingface.co/barghavani/English_to_German/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/10be16d0e5e5e3685ad1338c13bf0f5e023e101a23ffac075358811959890594.d1b447d2d77f64965845e2315488ade5473578846e926f3195a1a9e440231045
Model config T5Config {
  "_name_or_path": "barghavani/English_to_German",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

storing https://huggingface.co/barghavani/English_to_German/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/35f1f68337129534975b601befdd12bb0d7710204b615fb5d8486d09d3f19e14.ecbec2c4e05b5502ef0d307dd87fb6ec0ba140708be8a3c5ee03d176ea031677
creating metadata file for /root/.cache/huggingface/transformers/35f1f68337129534975b601befdd12bb0d7710204b615fb5d8486d09d3f19e14.ecbec2c4e05b5502ef0d307dd87fb6ec0ba140708be8a3c5ee03d176ea031677
loading weights file https://huggingface.co/barghavani/English_to_German/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/35f1f68337129534975b601befdd12bb0d7710204b615fb5d8486d09d3f19e14.ecbec2c4e05b5502ef0d307dd87fb6ec0ba140708be8a3c5ee03d176ea031677
All model checkpoint weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at barghavani/English_to_German.
If your task is similar to the task the 

...

storing https://huggingface.co/barghavani/English_to_German/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/3d1e6242a96eedca0cef1652f946618d9dade4c08cc15e708c729ab6afc6b0d3.c087d72eed7d6dc98a9190760501aad60aadd82a0ad8362e2be88c2679a08533
creating metadata file for /root/.cache/huggingface/transformers/3d1e6242a96eedca0cef1652f946618d9dade4c08cc15e708c729ab6afc6b0d3.c087d72eed7d6dc98a9190760501aad60aadd82a0ad8362e2be88c2679a08533
https://huggingface.co/barghavani/English_to_German/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpdw0zsv7n


Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

storing https://huggingface.co/barghavani/English_to_German/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/2a6df09d6ebd08c08700a9520f928017b81a330f9a253d08a6acfaae3f64f17f.d0682c01761a965a5741c21a60e6682035feecb9474e3981d3dc992c2b5a09a7
creating metadata file for /root/.cache/huggingface/transformers/2a6df09d6ebd08c08700a9520f928017b81a330f9a253d08a6acfaae3f64f17f.d0682c01761a965a5741c21a60e6682035feecb9474e3981d3dc992c2b5a09a7
https://huggingface.co/barghavani/English_to_German/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpgj895zhu


Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

storing https://huggingface.co/barghavani/English_to_German/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/784254d1de054fda59613ada46510d8c9eae44735995f73ed2c836cb42a841fc.c94798918c92ded6aeef2d2f0e666d2cc4145eca1aa6e1336fde07f2e13e2f46
creating metadata file for /root/.cache/huggingface/transformers/784254d1de054fda59613ada46510d8c9eae44735995f73ed2c836cb42a841fc.c94798918c92ded6aeef2d2f0e666d2cc4145eca1aa6e1336fde07f2e13e2f46
loading file https://huggingface.co/barghavani/English_to_German/resolve/main/spiece.model from cache at None
loading file https://huggingface.co/barghavani/English_to_German/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/2a6df09d6ebd08c08700a9520f928017b81a330f9a253d08a6acfaae3f64f17f.d0682c01761a965a5741c21a60e6682035feecb9474e3981d3dc992c2b5a09a7
loading file https://huggingface.co/barghavani/English_to_German/resolve/main/added_tokens.json from cache at None
loading file https://hugging

In [29]:
text = "translate English to French: Hi this is Bahareh."


In [30]:
translator_en_German(text)


[{'translation_text': 'das ist bahareh.'}]

In [31]:
translator_en_French(text)

[{'translation_text': "Bonjour, c'est Bahareh."}]